In [1]:
from newspaper import Article
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
import time as tm
import re
from bs4 import BeautifulSoup

In [10]:
cleaned = pd.read_csv('cleaned1.csv')

,Unnamed: 0,date,domain,url,text,cleaned,title,published,author
0,0,2019-10-08,www.newarkblack.com,http://www.newarkblack.com,NewarkBlack.com - Newark African American News...,0,NaN,September 25 2019,NaN
1,1,2019-10-08,www.newarkblack.com,http://www.newarkblack.com,�,0,NaN,September 25 2019,NaN
2,2,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/?s={search_term_str...,You searched for {search_term_string} - Newark...,0,{search_term_string} - search results,October 7 2019,NaN
3,3,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/advertise,Advertise - NewarkBlack.com CLOSE Home Adverti...,0,NaN,October 7 2019,NaN
4,4,2019-10-08,www.newarkblack.com,http://www.newarkblack.com/category/news,News Archives - NewarkBlack.com CLOSE Home Adv...,0,News,October 3 2019,WA Public Relations Company
...,...,...,...,...,...,...,...,...,...
2374,2374,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/category/food,Food Archive My Paper Online Home Disclaimer ...,0,NaN,NaN,new_view_media
2376,2376,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/category/nightlife,Nightlife Archive My Paper Online Home Discla...,0,NaN,December 22 2015,new_view_media
2393,2393,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/category/local-n...,Local News Archive My Paper Online Home Discl...,0,NaN,NaN,new_view_media
2399,2399,2019-11-02,www.mypaperonline.com,https://www.mypaperonline.com/category/local-n...,Local News Archive My Paper Online Home Discl...,0,NaN,NaN,new_view_media


In [8]:
article = Article('https://wayback.archive-it.org/12706/20200113151841/https://newbrunswicktoday.com/2019/06/12/under-pressure-rutgers-releases-rupd-and-nbpd-dispatch-logs/')
#article.html = str(f.read(), 'utf-8')
f.close()
article.download()
article.parse()
print(article.authors)
print(article.publish_date)
print(article.title)
print('\n')
print(article.text)

['Charlie Kratovil', 'About The Writer', 'More Writer', 'November', 'On', 'In', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav.Molongui-Author-Box-Tabs-Top Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav Label Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Input Id', "'Mab-Tab-Profile-'"]
2019-06-12 00:00:00
Under Pressure, Rutgers Releases RUPD and NBPD Dispatch Logs


NEW BRUNSWICK, NJ—Rutgers University released 1,272 pages of police records after being served with an amended complaint in a public records lawsuit filed by this reporter.

The Computer Aided Dispatch (CAD) logs list each and every call that the city's two major police departments responded to between February 1 and May 31, plus other instances where officers were dispatched.

After eighteen weeks of stalling, Rutgers released the records about 30 hours after being served with the amended complaint.

Then, the following day, Rutgers released an additional 299 pages consis

In [37]:
chosen.date[0]

Timestamp('2019-10-08 00:00:00')

In [54]:

# this creates and sets all the new variables to None
chosen['cleaned'] = None
chosen['title'] = None
chosen['published'] = None
chosen['author'] = None
chosen['date'] = pd.to_datetime(chosen['date'])

# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def get_html(collection_id, data):

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for index, row in data.iterrows():

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + row.url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:
    
            if dt.strptime(a.text, '%b %d, %Y') == row.date:
                
                article = Article('https:' + a.attrs['href'])

                try:
                    data.loc[index, 'title'] = article.title
                    data.loc[index, 'author'] = article.authors[0]
                    data.loc[index, 'published'] = article.published_date
                    data.loc[index, 'text'] = article.text
                    data.loc[index, 'cleaned'] = True
                except:
                    data.loc[index, 'cleaned'] = False

    return data

In [57]:
start = tm.time()
new_parse = get_html(12706, chosen)
span = tm.time() - start
print('completed in ', span, ' seconds')

completed in  879.2199304103851  seconds


In [59]:
new_parse.to_csv('test_parsing.csv')